# LAYERS.ipynb 
# Visualization Tools for Analysis

---
## Overview
- points.py
- polygons.py

# 1 - points.py

This file is responsible for generating the point layer.

# 1.1 - Imports

This section is responsible for necessary imports.

Annotations/type hints are imported for clarity. Pandas is imported for data management. Folium is used for generating the interactive map. MarkerCluster is used to group nearby markers for readability.

In [ ]:
from __future__ import annotations
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# 1.2 - Add Point Layer

This function adds a cluster point layer to a Folium map. Each point has a tooltip. The function takes a folium.Map `m`, a Pandas dataframe `points_df`, a string `name` (name that appears in the LayerControl toggle), a list of strings `tooltip_cols` (columns to include in the tooltip), and a string `color` (color of points/markers).

Line-by-line breakdown:
- Initialize marker cluster layer that allows individual markers to apepar at zoom >=13.
- Add cluster to map so that markers actually render.
- Iterate over points, converting each row into a dictionary. Retrieve latitude and longitude values and store them, while skipping to the next row if either value is NaN. Initialize tooltip list, then loop throw each tooltip column from the input argument. If the column appears in the point row dictionary and the entry is not None, then append the key-value pair to the tooltip list. Afterwards, create tooltip with appropriate line breaks (or just name of cluster if no tooltip data exists). Lastly, create a CircleMarker and add it to the cluster of markers.

In [ ]:
def add_point_layer(
    m: folium.Map,
    points_df: pd.DataFrame,
    name: str,
    tooltip_cols: list[str],
    color: str,
):
    cluster = MarkerCluster(name=name, disableClusteringAtZoom=13)
    cluster.add_to(m)

    for row in points_df.itertuples(index=False):
        d = row._asdict()
        lat, lon = d.get("lat"), d.get("lon")
        if pd.isna(lat) or pd.isna(lon):
            continue

        tooltip_parts = []
        for c in tooltip_cols:
            if c in d and d[c] is not None:
                tooltip_parts.append(f"{c}: {d[c]}")
        tooltip = "<br>".join(tooltip_parts) if tooltip_parts else name

        folium.CircleMarker(
            location=[float(lat), float(lon)],
            radius=3.5,
            color=color,          # outline
            fill=True,
            fill_color=color,     # fill
            fill_opacity=0.85,
            opacity=0.9,
            tooltip=folium.Tooltip(tooltip, sticky=True),
        ).add_to(cluster)



# 2 - polygons.py 

This file is responsible for implementing map layers as polygons, allowing users to easily identify food deserts and food swamps at tract-level (and use county-level boundaries as reference). 

# 2.1 - Imports

This section covers the necessary imports. 

Annotations/type hints are imported for clarity. Path is imported for file system management. GeoPandas and Pandas are used for data management. Folium is used for interactive mapping.

In [ ]:
from __future__ import annotations
from pathlib import Path
import geopandas as gpd
import pandas as pd
import folium

# 2.2 - Add Food Desert Layer

This function adds an unweighted food desert severity polygon layer. The function accepts a folium.Map `m`, a Path `tracts_geojson`, and a Pandas dataframe `desert_df`. 

Line-by-line breakdown:
- Read census tract polygons from GeoJSON and produce GeoDataFrame.
- Join desert scores (left join to keep all tracts, and missing scores automatically receive a 0 to prevent crashing)
- Scores are normalized based on the worst tract.
- Define a style function that accepts a `feat`, which in this case is a GeoJSON feature dictionary. This function stores severity from a certain `feat` in the variable `sev`.
- If `sev` is 0 or negative, then the polygon is rendered invisible and returned.
- Otherwise, severity and opacity are normalized and the resulting polygon is returned.
- Create a folium.GeoJson object using the merged table and the style function defined within the function, and add to the map.

In [ ]:
def add_food_desert_layer(m: folium.Map, tracts_geojson: Path, desert_df: pd.DataFrame):
    gdf = gpd.read_file(tracts_geojson)
    merged = (
        gdf.merge(desert_df, on="GEOID", how="left")
        .fillna({"desert_severity": 0})
    )

    # Compute max severity for normalization (avoid division by zero)
    max_sev = max(merged["desert_severity"].max(), 1)

    def style_fn(feat):
        sev = feat["properties"].get("desert_severity", 0) or 0

        if sev <= 0:
            return {
                "fillOpacity": 0.0,
                "weight": 0.2,
                "opacity": 0.05,
            }

        # Normalize severity → opacity (0.2 → 0.75)
        norm = sev / max_sev
        fill_opacity = 0.2 + 0.55 * norm

        return {
            "fillColor": "#d73027",  # red
            "color": "#b22222",      # darker red outline
            "fillOpacity": fill_opacity,
            "weight": 0.6,
            "opacity": 0.4,
        }

    folium.GeoJson(
        data=merged.__geo_interface__,
        name="Food Deserts (polygons)",
        style_function=style_fn,
        tooltip=folium.GeoJsonTooltip(
            fields=["GEOID", "population", "desert_severity"],
            aliases=["Census Tract", "Population", "Desert Severity"],
        )

    ).add_to(m)

# 2.3 - Add Food Swamp Layer

This function is responsible for adding unweighted food swamp index polygons to a map. The function accepts a folium.Map `m`, a Path `tracts_geojson`, and a Pandas dataframe `swamp_df`.

Line-by-line breakdown:
- Read in tracts data as `gdf`.
- Join tract dataframe using only necessary columns and ensure all invalid index scores are set to 0 to prevent crashing.
- Extreme tracts are capped at the 95th percentile to avoid flattening the scale.
- Style function (works similarly to the one in the food desert function 2.2, except food swamps are yellow instead of red).
- Create a folium.GeoJson object using the merged table and the style function defined within the function, and add to the map.

In [ ]:
def add_food_swamp_layer(m: folium.Map, tracts_geojson: Path, swamp_df: pd.DataFrame):
    gdf = gpd.read_file(tracts_geojson)
    merged = (
        gdf.merge(swamp_df[["GEOID", "swamp_index"]], on="GEOID", how="left")
        .fillna({"swamp_index": 0.0})
    )

    # Cap extreme swamp values to stabilize color scaling
    cap = merged["swamp_index"].quantile(0.95)
    cap = max(cap, 1.0)

    def style_fn(feat):
        v = float(feat["properties"].get("swamp_index", 0.0) or 0.0)

        if v <= 0:
            return {
                "fillOpacity": 0.0,
                "weight": 0.2,
                "opacity": 0.05,
            }

        # Normalize and clamp
        norm = min(v / cap, 1.0)

        # Opacity range: 0.2 → 0.7
        fill_opacity = 0.2 + 0.5 * norm

        return {
            "fillColor": "#fee08b",  # yellow
            "color": "#d9a400",      # darker yellow outline
            "fillOpacity": fill_opacity,
            "weight": 0.6,
            "opacity": 0.4,
        }

    folium.GeoJson(
        data=merged.__geo_interface__,
        name="Food Swamps (polygons)",
        style_function=style_fn,
        tooltip=folium.GeoJsonTooltip(
            fields=["GEOID", "population", "swamp_index"],
            aliases=["Census Tract", "Population", "Food Swamp Index"],
        )

    ).add_to(m)

# 2.4 - Add Population-Weighted Food Desert Layer

This function is nearly identical to the unweighted version, but assumes that `df` has a column `pop_weighted_desert`.

In [ ]:
def add_pop_weighted_food_desert_layer(m, tracts_geojson: Path, df: pd.DataFrame):
    gdf = gpd.read_file(tracts_geojson)
    merged = gdf.merge(
        df[["GEOID", "pop_weighted_desert"]],
        on="GEOID",
        how="left"
    ).fillna({"pop_weighted_desert": 0})

    cap = merged["pop_weighted_desert"].quantile(0.95)
    cap = max(cap, 1)

    def style_fn(feat):
        v = feat["properties"].get("pop_weighted_desert", 0)
        if v <= 0:
            return {"fillOpacity": 0.0, "weight": 0.2, "opacity": 0.05}

        norm = min(v / cap, 1.0)
        fill_opacity = 0.25 + 0.55 * norm

        return {
            "fillColor": "#d73027",
            "color": "#b22222",
            "fillOpacity": fill_opacity,
            "weight": 0.6,
            "opacity": 0.4,
        }

    folium.GeoJson(
        merged.__geo_interface__,
        name="Food Deserts (population-weighted)",
        style_function=style_fn,
        tooltip=folium.GeoJsonTooltip(
            fields=["GEOID", "pop_weighted_desert"],
            aliases=["Census Tract", "Population-weighted desert impact"],
        ),
    ).add_to(m)

# 2.5 - Add Population-Weighted Food Swamp Layer

This function is nearly identical to the unweighted version, but assumes that `df` has a column `pop_weighted_swamp`.

In [ ]:
def add_pop_weighted_food_swamp_layer(m, tracts_geojson: Path, df: pd.DataFrame):
    gdf = gpd.read_file(tracts_geojson)
    merged = gdf.merge(
        df[["GEOID", "pop_weighted_swamp"]],
        on="GEOID",
        how="left"
    ).fillna({"pop_weighted_swamp": 0})

    cap = merged["pop_weighted_swamp"].quantile(0.95)
    cap = max(cap, 1)

    def style_fn(feat):
        v = feat["properties"].get("pop_weighted_swamp", 0)
        if v <= 0:
            return {"fillOpacity": 0.0, "weight": 0.2, "opacity": 0.05}

        norm = min(v / cap, 1.0)
        fill_opacity = 0.25 + 0.5 * norm

        return {
            "fillColor": "#fee08b",
            "color": "#d9a400",
            "fillOpacity": fill_opacity,
            "weight": 0.6,
            "opacity": 0.4,
        }

    folium.GeoJson(
        merged.__geo_interface__,
        name="Food Swamps (population-weighted)",
        style_function=style_fn,
        tooltip=folium.GeoJsonTooltip(
            fields=["GEOID", "pop_weighted_swamp"],
            aliases=["Census Tract", "Population-weighted swamp impact"],
        ),
    ).add_to(m)

# 2.6 - Add County Boundaries

This function adds county reference boundaries to allow users to make comparisons at a county-level instead of just a tract-level. The function accepts a folium.Map `m` and a Path `counties_geojson`.

Line-by-line breakdown:
- County data is read into a GeoDataFrame.
- folium.GeoJson creates actual boundary overlay with tooltip, and the layer is added to the map.

In [ ]:
def add_county_boundaries(m: folium.Map, counties_geojson: Path):
    gdf = gpd.read_file(counties_geojson)

    folium.GeoJson(
        gdf.__geo_interface__,
        name="County boundaries",
        style_function=lambda _: {
            "fillOpacity": 0.0,
            "color": "#2b2b2b",
            "weight": 1.2,
            "opacity": 0.6,
        },
        tooltip=folium.GeoJsonTooltip(fields=["NAME"], aliases=["County"]),
    ).add_to(m)
